# 6.1_Extraer_Caracteristicas

In [ ]:
#!pip install scikit-learn
#!pip install imutils
#!pip install progressbar

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from pyimagesearch.io import HDF5DatasetWriter
from imutils import paths
import numpy as np
import progressbar
import argparse
import random
import os

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
img_size = 224
batch_size = 32
buffer_size = 1000 #control the number of extracted feature store in memory
path_train = 'data_' + str(img_size) + '/train'
path_validation = 'data_' + str(img_size) + '/validation'
path_output_train_hdf5 = 'HDF5/MobileNetV2_train.hdf5'
path_output_validation_hdf5 = 'HDF5/MobileNetV2_validation.hdf5'

In [4]:
#cargando las imágenes
imagePaths_train = list(paths.list_images(path_train))
imagePaths_validation = list(paths.list_images(path_validation))
#moviendo de forma aleatoria el orden de las imágenes
random.shuffle(imagePaths_train)
random.shuffle(imagePaths_validation)

#Obteniendo la etiqueta de la imagen
labels_train = [p.split(os.path.sep)[-2] for p in imagePaths_train]
#Códificando las etiquetas en númerops
le = LabelEncoder()
labels_train = le.fit_transform(labels_train)

#Obteniendo la etiqueta de la imagen
labels_validation = [p.split(os.path.sep)[-2] for p in imagePaths_validation]
#Códificando las etiquetas en númerops
le = LabelEncoder()
labels_validation = le.fit_transform(labels_validation)

# creando el modelo base

In [5]:
model = MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(img_size,img_size,3)
)

# Grabando en disco las caracteristicas

In [6]:
dataset_train = HDF5DatasetWriter((len(imagePaths_train), 1280 * 7 * 7), #es igual al tamaño de la última capa
                            path_output_train_hdf5,
                            dataKey = "features",
                            bufSize = buffer_size)
dataset_train.storeClassLabels(le.classes_)

dataset_validation = HDF5DatasetWriter((len(imagePaths_validation), 1280 * 7 * 7), #es igual al tamaño de la última capa
                            path_output_validation_hdf5,
                            dataKey = "features",
                            bufSize = buffer_size)
dataset_validation.storeClassLabels(le.classes_)

# Obteniendo las caracteristicas

In [7]:
widgets = ["Extracción de características: ",
           progressbar.Percentage(),
           " ",
           progressbar.Bar(),
           " ",
           progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=len(imagePaths_train),
                               widgets=widgets).start()

# recorriendo el array con una distancia del batch_size
for i in np.arange(0, len(imagePaths_train), batch_size):
    # obteniendo los paths y labels de las imágenes
    batchPaths = imagePaths_train[i:i + batch_size]
    batchLabels = labels_train[i:i + batch_size]
    batchImages = []
    # recorremos el bath obtenido
    for (j, imagePath) in enumerate(batchPaths):
        #cargamos las imagenes con keras
        image = load_img(imagePath, target_size=(img_size, img_size))
        image = img_to_array(image)
        
        #hacemos un pre-procesamiento de la imagen
        #le agregamos una dimensión
        image = np.expand_dims(image, axis=0)
        #obteniendo la media de la intesidad RGB del pixel
        image = imagenet_utils.preprocess_input(image)
        
        #agregando la imagen al batch
        batchImages.append(image)
        
    #creamos un stack con el array
    batchImages = np.vstack(batchImages)
    features = model.predict(batchImages, batch_size=batch_size)
    
    #hacemos un reshape de la predicción, convirtiendolo en 1D
    features = features.reshape((features.shape[0], 1280 * 7 * 7))
    
    #agregamps los features en  el HDF5
    dataset_train.add(features, batchLabels)
    pbar.update(i)
        
#cerramos el  HDF5
dataset_train.close()
pbar.finish()

Extracción de características: 100% |###########################| Time: 0:00:10


In [8]:
widgets = ["Extracción de características: ",
           progressbar.Percentage(),
           " ",
           progressbar.Bar(),
           " ",
           progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=len(imagePaths_validation),
                               widgets=widgets).start()

# recorriendo el array con una distancia del batch_size
for i in np.arange(0, len(imagePaths_validation), batch_size):
    # obteniendo los paths y labels de las imágenes
    batchPaths = imagePaths_validation[i:i + batch_size]
    batchLabels = labels_validation[i:i + batch_size]
    batchImages = []
    # recorremos el bath obtenido
    for (j, imagePath) in enumerate(batchPaths):
        #cargamos las imagenes con keras
        image = load_img(imagePath, target_size=(img_size, img_size))
        image = img_to_array(image)
        
        #hacemos un pre-procesamiento de la imagen
        #le agregamos una dimensión
        image = np.expand_dims(image, axis=0)
        #obteniendo la media de la intesidad RGB del pixel
        image = imagenet_utils.preprocess_input(image)
        
        #agregando la imagen al batch
        batchImages.append(image)
        
    #creamos un stack con el array
    batchImages = np.vstack(batchImages)
    features = model.predict(batchImages, batch_size=batch_size)
    
    #hacemos un reshape de la predicción, convirtiendolo en 1D
    features = features.reshape((features.shape[0], 1280 * 7 * 7))
    
    #agregamps los features en  el HDF5
    dataset_validation.add(features, batchLabels)
    pbar.update(i)
        
#cerramos el  HDF5
dataset_validation.close()
pbar.finish()

Extracción de características: 100% |###########################| Time: 0:00:08


# Entrenando el clasificador

In [16]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import argparse
import pickle
import h5py

In [12]:
#leemos el HDF5
db_train = h5py.File(path_output_train_hdf5, 'r')
db_validation = h5py.File(path_output_validation_hdf5, 'r')

In [13]:
print(db_train["features"].shape)
print(db_train["labels"].shape)

(2240, 62720)
(2240,)


In [14]:
# entrenamos nuestro modelo usando las caracteristicas 128-d
# luego reproduce el reconocimiento facial
print("[INFO] entrenando el modelo...")
recognizer = SVC(C=1.0, kernel="linear", probability=True)
recognizer.fit(db_train["features"], db_train["labels"])

[INFO] entrenando el modelo...


SVC(kernel='linear', probability=True)

# Evaluando el clasificador

In [17]:
preds = recognizer.predict(db_validation["features"])
print(classification_report(db_validation["labels"], preds, target_names=db_validation["label_names"]))

              precision    recall  f1-score   support

          10       0.23      0.28      0.26       477
         100       0.34      0.29      0.31       584
          20       0.21      0.48      0.29       485
         200       0.55      0.27      0.36       583
          50       0.51      0.17      0.25       542

    accuracy                           0.29      2671
   macro avg       0.37      0.30      0.29      2671
weighted avg       0.38      0.29      0.30      2671



In [ ]:

model_output = 'machine_learning/svc_mobilenet.pickle'
labels_ouput = 'machine_learning/svc_labels.pickle'

# grabamos en disco nuestro modelo
f = open(model_output, "wb")
f.write(pickle.dumps(recognizer))
f.close()
# write the label encoder to disk
f = open(labels_ouput, "wb")
f.write(pickle.dumps(le))
f.close()
